In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import math

import numpy as np
import pandas as pd
import torch
from dataset import Collator, Dataset
from metrics import compute_metrics_on_df
from nn_modules.poolers import MeanPooler
from nn_modules.models import SiameseManhattanBERT
from sklearn.model_selection import train_test_split
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm, trange
from train import train
from transformers import AutoModel, AutoTokenizer
from utils import chunks, set_global_seed

In [ ]:
# import sys
# sys.path.append("../../pytorch-trainer")

# from pytorch_autotrainer import Trainer

In [ ]:
# reproducibility
set_global_seed(42)

In [ ]:
# parameters
config = {
    'MODEL_NAME':    'distilroberta-base',
    'BATCH_SIZE':    128,
    'LEARNING_RATE': 1e-5,
    'N_EPOCHS':      5,
    'LOSS':          'mse',
}

In [ ]:
# tensorboard
experiment_name = f"MODEL_{config['MODEL_NAME']}_BATCH_{config['BATCH_SIZE']}_LR_{config['LEARNING_RATE']}_LOSS_{config['LOSS']}"

writer = SummaryWriter(
    log_dir=f"runs/{experiment_name}",
)

In [ ]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

### prepare data

In [ ]:
df = pd.read_csv('./data/train.csv', index_col='id')
df.fillna('', inplace=True)

df['question1'] = df['question1'].str.lower()
df['question2'] = df['question2'].str.lower()

In [ ]:
df_submission = pd.read_csv('./data/test.csv', index_col='test_id')
df_submission.fillna('', inplace=True)

df_submission['question1'] = df_submission['question1'].str.lower()
df_submission['question2'] = df_submission['question2'].str.lower()

In [ ]:
df

In [ ]:
df_submission

In [ ]:
df['is_duplicate'].value_counts()

In [ ]:
df_train, df_test = train_test_split(
    df,
    test_size=0.25,
    random_state=42,
    shuffle=True,
    stratify=df['is_duplicate'],
)

In [ ]:
print(f'Train size: {df_train.shape[0]}')
print(f'Test size: {df_test.shape[0]}')

In [ ]:
train_dataset = Dataset(df=df_train)
test_dataset = Dataset(df=df_test)

### load bert model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(config['MODEL_NAME'])
bert_model = AutoModel.from_pretrained(config['MODEL_NAME'])

In [ ]:
tokenizer_kwargs = {
    'return_tensors': 'pt',
    'padding':        True,
    'truncation':     True,
    'max_length':     512,
}

collate_fn = Collator(tokenizer, tokenizer_kwargs)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=config['BATCH_SIZE'],
    shuffle=True,
    collate_fn=collate_fn,
)
test_dataloader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=config['BATCH_SIZE'],
    shuffle=False,
    collate_fn=collate_fn,
)

### train model

In [ ]:
pooler = MeanPooler()

model = SiameseManhattanBERT(
    bert_model=bert_model,
    pooler=pooler,
).to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=config['LEARNING_RATE'])
criterion = torch.nn.MSELoss()

In [ ]:
train(
    n_epochs=config['N_EPOCHS'],
    model=model,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    optimizer=optimizer,
    criterion=criterion,
    writer=writer,
    device=device,
)

# trainer = Trainer(
#     model=model,
#     optimizer=optimizer,
#     compute_metrics=compute_metrics_on_df,  # TODO: maybe make optional
#     experiment_name=experiment_name,
# )

In [ ]:
torch.save(model.cpu().state_dict(), f'{experiment_name}.pth')

### evaluate model

In [ ]:
model = SiameseManhattanBERT(
    bert_model=bert_model,
    pooler=pooler,
)

model.load_state_dict(torch.load(f'{experiment_name}.pth'))
model.to(device)
model.eval();

In [ ]:
train_metrics = compute_metrics_on_df(
    model=model,
    df=df_train,
    tokenizer=tokenizer,
    tokenizer_kwargs=tokenizer_kwargs,
    batch_size=config['BATCH_SIZE'],
)

In [ ]:
train_metrics

In [ ]:
test_metrics = compute_metrics_on_df(
    model=model,
    df=df_test,
    tokenizer=tokenizer,
    tokenizer_kwargs=tokenizer_kwargs,
    batch_size=config['BATCH_SIZE']
)

In [ ]:
test_metrics

### submission

In [ ]:
length = math.ceil(len(df_submission) / config['BATCH_SIZE'])

In [ ]:
q1_emb = []
for texts in tqdm(
    chunks(df_submission['question1'].to_list(), n=config['BATCH_SIZE']),
    total=length,
    desc='vectorize question1',
):
    emb = model.vectorize(texts, tokenizer, tokenizer_kwargs)
    q1_emb.append(emb)

q2_emb = []
for texts in tqdm(
    chunks(df_submission['question2'].to_list(), n=config['BATCH_SIZE']),
    total=length,
    desc='vectorize question2',
):
    emb = model.vectorize(texts, tokenizer, tokenizer_kwargs)
    q2_emb.append(emb)

In [ ]:
y_score = []

for i in trange(length):
    y_score_batch = model.exponent_neg_manhattan_distance(q1_emb[i], q2_emb[i], type='np')
    y_score.append(y_score_batch)

y_score = np.concatenate(y_score)

In [ ]:
df_submission['is_duplicate'] = y_score
df_submission['is_duplicate'].to_csv('submission.csv')